In [1]:
import torch
import torch.nn as nn
import random
from torch.utils.tensorboard import SummaryWriter
import pandas as pd

class NeuralNetworkWithFixedOutput(nn.Module):
    def __init__(self, genome):
        super(NeuralNetworkWithFixedOutput, self).__init__()
        
        # Creazione dinamica degli strati intermedi in base al genoma
        self.layers = nn.ModuleList()
        for i in range(len(genome)):
            input_size, output_size, weights, bias = genome[i]
            layer = nn.Linear(input_size, output_size)
            
            # Impostazione dei pesi e del bias
            with torch.no_grad():
                layer.weight = nn.Parameter(torch.tensor(weights, dtype=torch.float32))
                layer.bias = nn.Parameter(torch.tensor(bias, dtype=torch.float32))
            
            self.layers.append(layer)

        # Aggiunta dell'ultimo strato fisso per l'output di 44 neuroni
        self.output_layer = nn.Linear(output_size, 44)

    def forward(self, x):
        # Forward pass attraverso ogni strato intermedio
        for layer in self.layers:
            x = torch.relu(layer(x))  # Utilizzo di ReLU come funzione di attivazione

        # Passaggio attraverso lo strato di output e applicazione della Softmax
        x = self.output_layer(x)
        return torch.softmax(x, dim=0)


def generate_random_genome(input_size=25, output_size=44, min_hidden_layers=2, max_hidden_layers=5):
    num_hidden_layers = random.randint(min_hidden_layers, max_hidden_layers)

    # Struttura del genoma: [(input_size, output_size, weights, bias), ...]
    genome = []
    
    # Dimensioni dello strato precedente, inizia con la dimensione dell'input
    previous_layer_size = input_size

    # Generazione casuale degli strati nascosti
    for _ in range(num_hidden_layers):
        current_layer_size = random.randint(1, 10)  # Numero casuale di neuroni nello strato

        # Pesi casuali e bias per lo strato corrente
        weights = [[random.uniform(-1, 1) for _ in range(previous_layer_size)] for _ in range(current_layer_size)]
        bias = [random.uniform(-1, 1) for _ in range(current_layer_size)]

        # Aggiungi lo strato al genoma
        genome.append((previous_layer_size, current_layer_size, weights, bias))

        # Aggiorna la dimensione dello strato precedente
        previous_layer_size = current_layer_size
        
    # Ultimo strato che collega all'output fisso di 44 neuroni
    weights = [[random.uniform(-1, 1) for _ in range(previous_layer_size)] for _ in range(output_size)]
    bias = [random.uniform(-1, 1) for _ in range(output_size)]
    genome.append((previous_layer_size, output_size, weights, bias))

    return genome


In [55]:
random_genome = generate_random_genome()

In [50]:
df = pd.DataFrame(random_genome, columns=['input_size', 'output_size', 'weights', 'bias'])
print(len(df["weights"][0][0]))
df

25


,input_size,output_size,weights,bias
0,25,9,"[[-0.3117761836346904, -0.5561000708446451, 0....","[0.18014114837596185, -0.36997391845156, -0.69..."
1,9,4,"[[-0.00037122968064595696, -0.1965871505247158...","[-0.35258279596613074, 0.9327769710380127, -0...."
2,4,5,"[[-0.7568930576038975, -0.5751388755582068, -0...","[0.9030318588893254, -0.9340022951960045, -0.2..."
3,5,6,"[[-0.4502914407789407, 0.23211061245532227, 0....","[-0.9851832008989692, 0.3179269156330635, -0.8..."
4,6,44,"[[-0.2531974816365161, 0.33321756710870765, -0...","[-0.3052827094060304, -0.7238768419060704, 0.7..."


In [56]:
# Creazione della rete neurale
net = NeuralNetworkWithFixedOutput(random_genome)

# Test con un input casuale
test_input = torch.rand(25)
output = net(test_input)

print(output) 
print(output.shape)  # Output e la sua lunghezza (dovrebbe essere 44)

# Inizializzazione di TensorBoard
writer = SummaryWriter("runs/neural_network_visualization")

# Aggiunta della rete a TensorBoard
writer.add_graph(net, test_input)
writer.close()

tensor([0.0161, 0.0368, 0.0228, 0.0378, 0.0187, 0.0251, 0.0415, 0.0127, 0.0126,
        0.0113, 0.0146, 0.0352, 0.0366, 0.0135, 0.0204, 0.0144, 0.0104, 0.0355,
        0.0255, 0.0192, 0.0224, 0.0094, 0.0286, 0.0154, 0.0251, 0.0314, 0.0215,
        0.0230, 0.0226, 0.0236, 0.0306, 0.0250, 0.0146, 0.0315, 0.0286, 0.0132,
        0.0177, 0.0345, 0.0107, 0.0136, 0.0148, 0.0368, 0.0262, 0.0189],
       grad_fn=<SoftmaxBackward0>)
torch.Size([44])
